In [2]:
# import all necessary objects and methods for quantum circuits
import cirq
import random

In [ ]:

message = "We are out of pizza"
# Get bytes of the message using UTF-8 encoding
bytes_message = message.encode('utf-8')

# Convert each byte to a binary string with leading zeros for 8 bits
binary_message = ''.join(format(b, "08b") for b in bytes_message)
n = len(binary_message)

# QKD part
Initial_key_lenght = int(4.3*n)
Alice_qubits = cirq.LineQubit.range(Initial_key_lenght) # quantum register with Initial_key_lenght qubits
# Quantum circuit for Alice state
Alice_circuit = cirq.Circuit()

Alice_initial_key=[] #Initial bit string to send
Alice_basis=[] #List to save information about encoding basis
Bob_basis=[] #List to save information about decoding basis

#Creating random bit string
binary_list = [0,1]
binary_weights = [0.5, 0.5]
# Alice chooses randomly a string of binary bits
Alice_initial_key = random.choices(binary_list, weights=binary_weights, k=Initial_key_lenght)
    
#Preparing qubits, apply X gate if bit is equal 1
for i, bit in enumerate(Alice_initial_key):
    if bit==1:
        Alice_circuit.append(cirq.X(Alice_qubits[i])) # apply x-gate

#Encoding
Encoding_basis = ['Z', 'X']
Alice_basis = random.choices(Encoding_basis, weights=binary_weights, k=Initial_key_lenght) #Alice randomly pick a string of basis 
for i, basis in enumerate(Alice_basis):
    if basis=='Z': #if basis is 'Z' , then she encodes the bit in Z basis(do nothing)
        pass
    else: #if basis is 'X' , then she encodes the bit in X basis
        Alice_circuit.append(cirq.H(Alice_qubits[i]))

print("Alice\'s initial key ", Alice_initial_key)
print('\nAlice\'s randmoly chosen bases: ', Alice_basis)
#print('\nAlice\'s circuit:\n', Alice_circuit)



In [ ]:
# Quantum circuit for Bob's state
Bob_circuit = cirq.Circuit()

Bob_basis = random.choices(Encoding_basis, weights=binary_weights, k=Initial_key_lenght) #Bob randomly pick a string of basis
# Based on the randomly chosen basis, Bob apply that gate so can be able to measuremnt in the appropriate basis
for i, basis in enumerate(Bob_basis):
    if basis=='Z': #if basis is 'Z'
        pass 
    else: ##if basis is 'X'
        Bob_circuit.append(cirq.H(Alice_qubits[i]))
        

# Bob measures the recieving qubits        
Bob_circuit.append(cirq.measure(*Alice_qubits, key='Bob key'))
# the whole circuit
BB84_circuit = Alice_circuit + Bob_circuit

# simulating the BB84 protocol  
n_rep = 100
sim = cirq.Simulator()
results = sim.run(BB84_circuit, repetitions=n_rep)
Bob_initial_key = results.measurements['Bob key'][0]



print("Alice sent:", Alice_initial_key)
print("Alice encoding basis:", Alice_basis)
print("Bob received:", Bob_initial_key)
print("Bob decoding basis:", Bob_basis)

In [ ]:
#Sifting
Alice_final_key=[] #Alice list for matching basis
Bob_final_key=[] #Bob list for matching basis
for j in range(len(Alice_basis)): #Going through list of bases 
    if Alice_basis[j] == Bob_basis[j]: #Comparing
        #Keeping key bit if bases matched
        Alice_final_key.append(Alice_initial_key[j])
        Bob_final_key.append(Bob_initial_key[j]) 
    else:
        pass #Discard round if bases mismatched

print("Alice\'s key =", Alice_final_key)
print("Bob key\'s =", Bob_final_key)

In [ ]:
#QBER
batch = len(Alice_final_key)//2    #To divide without remainer, use //
errors=0
for i in range(batch):
    bit_index = random.randrange(len(Alice_final_key)) 
    tested_bit = Alice_final_key[bit_index]
    print ("Alice randomly selected bit index =", bit_index, ", and its value is = ", tested_bit)
    if Alice_final_key[bit_index]!=Bob_final_key[bit_index]: #comparing tested rounds
        errors=errors+1 #calculating errors
    #removing tested bits from key strings
    del Alice_final_key[bit_index] #Use del to specify the index of the element you want to delete
    del Bob_final_key[bit_index]
QBER=errors/batch #calculating QBER
        
print("QBER value =", QBER)
print("Alice's secret key =", Alice_final_key)
print("Bob's secret key =", Bob_final_key)

In [96]:
#Encryption
# Use the generated shared key to encrypt the message using classical XOR encryption
def encrypt_message(message, key):
    encrypted_message = ''
    for i in range(len(message)):
        encrypted_bit = int(message[i]) ^ key[i]
        encrypted_message += str(encrypted_bit)
    return encrypted_message

encrypted_message = encrypt_message(binary_message, Alice_final_key)

In [97]:
# Decryption
def binary_to_str(binary_string, binary_length=8):
    # Convert binary string to bytes
    binary_bytes = bytes(int(binary_string[i:i+binary_length], 2) for i in range(0, len(binary_string), binary_length))
    text = binary_bytes.decode('utf-8')
    return text

# Use the shared key received from Alice to decrypt the message
def decrypt_message(encrypted_message, key):
    decrypted_message = ''
    for i in range(len(encrypted_message)):
        decrypted_bit = int(encrypted_message[i]) ^ key[i]
        decrypted_message += str(decrypted_bit)
    return decrypted_message

decrypted_message = decrypt_message(encrypted_message, Bob_final_key)
decrypted_message = binary_to_str(decrypted_message)




In [98]:
#  Verification

# Verify if the decrypted message matches the original message
if decrypted_message == message:
    print("Encryption and decryption successful!")
    print("Original message:", message)
    print("Decrypted message:", decrypted_message)
else:
    print("Encryption and decryption failed!")

Encryption and decryption successful!
Original message: We are out of pizza
Decrypted message: We are out of pizza
